In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import PorterStemmer as stemmer
from tqdm.notebook import tqdm
import rule_book_functs as rbfuncts

In [2]:
incidents_fn = "data/source/20220413_D1_Incidents.csv"
rule_book_fn = "data/rule_book.csv"

In [3]:
rules = pd.read_csv(rule_book_fn)
incidents = pd.read_csv(incidents_fn, dtype=str)

In [4]:
incidents.rename(columns={'IncidentNumber': 'incident_id'}, inplace=True)
incidents['text'] = (
        incidents['ShortDescription'].astype(str).fillna('') + ' ' + 
        incidents['FullDescription'].astype(str).fillna('') + ' ' + 
        incidents['ImmediateAction'].astype(str).fillna('')
).str.lower()

incidents = incidents[['incident_id', 'text']]

In [5]:
#sample100 = incidents.sample(n=100)
#sample100.index = range(100)
#sample100.to_csv('sample100.csv', index=False)
sample100 = pd.read_csv('sample100.csv', dtype=str)

In [6]:
#incidents.text[0]

In [7]:
#test = incidents.text.head(1).apply(rbfuncts.lemmatize_text)
#print(list(test))

In [8]:
#test = incidents.text.head(1).apply(rbfuncts.stem_text)
#print(list(test))

In [9]:
#test_tokens = rbfuncts.tokenize(incidents.text[0])

In [10]:
#chk_text = 'employee was grinding with a four-inch grinder on an iron support in the compressor building when he felt discomfort to his left eye.'
#pattern = '(\\bemployee (.*) discomfort (.*) eye\\b)'
#rbfuncts.check_presence(pattern, chk_text)

In [11]:
#sents = sent_tokenize(incidents.text[0])
#for sent in sents:
#    if rbfuncts.check_presence(pattern, sent):
#        print(f'{rbfuncts.check_presence(pattern, sent)}: {sent}')

## Load Synonyms & Rules

In [12]:
syn_csv = pd.read_csv('synonyms.csv')

# Load the dictionary of synonyms
syn_dict = {}
for r in range(len(syn_csv)):
    syn_toks = syn_csv.keywords[r].split(',')
    syn_dict.update({syn_csv.syn[r]:syn_toks})
    
rul_csv = pd.read_csv('rules.csv')

In [13]:
finds_df = rbfuncts.rule_book_scan(sample100, syn_dict, rul_csv)

Checking rule 1 of 24 (eye injury or irritation)
14/100 [0] 0.0 hrs 0.0% classified (14.0% through)...

TypeError: slice indices must be integers or None or have an __index__ method

In [14]:
#finds_df

In [15]:
strs = 'This is me. He tripped and fell from the ladder.'
#strs = 'during the scaffolding elements transporting for the assembly of scaffolding scaffolder slipped and fell due to the slippery floor'
test = pd.DataFrame({'incident_id':[1052], 'text':[strs]})
rbfuncts.rule_book_scan(test, syn_dict, rul_csv)

Checking rule 1 of 24 (eye injury or irritation)
1/1 [0] 0.0 hrs 0.0% classified (100.0% through)...

Checking rule 2 of 24 (eye injury or irritation)
1/1 [0] 0.0 hrs 0.0% classified (100.0% through)...

Checking rule 3 of 24 (back injury)
1/1 [0] 0.0 hrs 0.0% classified (100.0% through)...

Checking rule 4 of 24 (covid)
1/1 [0] 0.0 hrs 0.0% classified (100.0% through)...

Checking rule 5 of 24 (fall from or working at height issue)
1/1 [0] 0.0 hrs 0.0% classified (100.0% through)...

Checking rule 6 of 24 (fall from or working at height issue)
1/1 [0] 0.0 hrs 0.0% classified (100.0% through)...

Checking rule 7 of 24 (fall from or working at height issue)
1/1 [0] 0.0 hrs 0.0% classified (100.0% through)...

Checking rule 8 of 24 (foot or leg injury)
1/1 [1] 0.0 hrs 100.0% classified (100.0% through)...

Checking rule 9 of 24 (general injury)
1/1 [1] 0.0 hrs 100.0% classified (100.0% through)...

Checking rule 10 of 24 (hand or arm injury)
1/1 [1] 0.0 hrs 100.0% classified (100.0% thro

TypeError: expected string or bytes-like object

In [ ]:
print(not False)

In [ ]:
# How many incidents were classified
len(list(dict.fromkeys(finds_df[finds_df.finds_list == True].incid_nums)))

In [ ]:
rbfuncts.deepdive_results(sample100, incidents, finds_df, focus='finds')

In [ ]:
rbfuncts.deepdive_results(sample100, incidents, finds_df, focus='misses')

In [ ]:
chk = 'the country gm and regional gm were on their way to their apartment when they were involved in a motor vehicle collision.'
#check_presence('hello(.*)vehicle(.*)collision', chk)
#check_presence('\bregional (.*) were (.*) way\b', chk)
#bool(re.search('\bregional (.*) were (.*) way\b', chk))
bool(re.search('involved(.*)motor(.*)collision', chk))

In [ ]:
org_string = "This is employee's hand"
pattern = r"'s"

# Replace all occurrences of character 's with an empty string
org_string = re.sub(pattern, '', org_string )
print(org_string)

In [27]:
def check_presence(pattern, string):
    if pattern:
        return bool(re.search(pattern, string))
    else:
        return False

In [ ]:
rul_csv

In [ ]:
x = 'car(.*)hit(.*)\\bcat'
pattern = x
check_presence(pattern, "the car hit the cat")

In [ ]:
re.sub(r"\\\\b", "\\b", 'no\\\\b')

In [ ]:
test = pd.DataFrame({'id':[1, 2, 2, 2, 3], 'cats':['apple', 'apple', 'apple', 'orange', 'kiwi']})
test

In [ ]:
test.groupby(['id'])['cats'].apply(', '.join).reset_index()

In [ ]:
'apple, apple, orange'.split(', ')

In [ ]:
list(dict.fromkeys('apple, apple, orange'.split(', ')))

In [ ]:
", ".join(list(dict.fromkeys('apple, apple, orange'.split(', '))))

In [ ]:
def remove_dups(ent):
    split_ent = ent.split(', ')
    uniqe_ent = list(dict.fromkeys(split_ent))
    rjoin_ent = ", ".join(uniqe_ent)
    return(rjoin_ent)

In [ ]:
remove_dups('apple, apple, orange')

In [ ]:
test = test.groupby(['id'])['cats'].apply(', '.join).reset_index()
test['cats'] = test['cats'].apply(lambda x: remove_dups(x))

In [ ]:
test

In [ ]:
for r in [1, 2, 3]:
    print(r)

In [ ]:
finds_count = 0
finds_count+=1
finds_count

In [ ]:
from time import sleep
import sys

for i in range(200):
    sys.stdout.write('\r')
    # the exact output you're looking for:
    sys.stdout.write(str(i))
    sys.stdout.flush()
    sleep(0.05)

In [ ]:
from datetime import datetime

now = datetime.now()


In [ ]:
then = datetime.now()

In [ ]:
diff = then - now

In [ ]:
round(diff.seconds/(60*60), 2)

In [ ]:
round(60*60/(60*60), 2)

In [ ]:
value = 10000
f'{value:,}'

In [ ]:
test = pd.DataFrame({'rule':['employee(.*)hurt(.*)hand', 'ip(.*)hurt(.*)hand']})

In [ ]:
test

In [ ]:
test['status'] = test['rule'].apply(lambda x: check_presence(x, 'the ip hurt his right hand'))
test

In [17]:
# Load raw incident data
incidents = pd.read_csv("data/source/20220413_D1_Incidents.csv", dtype=str)  

# Load the synonym dictionary in its raw csv format
syn_csv = pd.read_csv('synonyms.csv')

# Load the rule definitions
rul_csv = pd.read_csv('rules.csv')

# Concatenate some of the fields to make the 'text' field for searching
incidents.rename(columns={'IncidentNumber': 'incident_id'}, inplace=True)
incidents['text'] = (
        incidents['ShortDescription'].astype(str).fillna('') + ' ' + 
        incidents['FullDescription'].astype(str).fillna('') + ' ' + 
        incidents['ImmediateAction'].astype(str).fillna('')
).str.lower()

# We only need the incident ID and the text for now
incidents = incidents[['incident_id', 'text']]

# Now convert the csv format into a dictionary of synonyms
syn_dict = {}
for r in range(len(syn_csv)):
    syn_toks = syn_csv.keywords[r].split(',')
    syn_dict.update({syn_csv.syn[r]:syn_toks})

In [18]:
rul_csv

,rule,keyword,syns,voids,span,shuffle
0,eye injury or irritation,eye,"worker_syns, pain_syns, eye_syns",NaN,8.0,True
1,eye injury or irritation,eye,"gen1_syns, pain_syns, eye_syns",NaN,8.0,True
2,back injury,back,"worker_syns, back_pain_syns, back_syns",NaN,8.0,True
3,covid,-,"worker_syns, tested_syns, covid_syns",NaN,8.0,True
4,fall from or working at height issue,height,"height_equipment_syns, falling_object_syns","same level, slippery floor, wet floor, slipper...",8.0,False
5,fall from or working at height issue,fall,"height_equipment_syns, falling_object_syns","same level, slippery floor, wet floor, slipper...",8.0,False
6,fall from or working at height issue,-,"worker_syns, height_equipment_syns, falling_ob...","same level, slippery floor, wet floor, slipper...",8.0,True
7,foot or leg injury,-,"worker_syns, hurt_syns, leg_syns",NaN,8.0,True
8,general injury,-,"worker_syns, hurt_syns, gen_body_syns",NaN,8.0,True
9,hand or arm injury,-,"worker_syns, hurt_syns, arm_syns",NaN,8.0,True


In [19]:
# worker_syns, pain_syns, eye_syns
worker_syns = ['employee', 'ip', 'worker', 'man', 'woman', 'he', 'she', 'his']
pain_syns = ['pain', 'discomfort', 'soreness', 'hurt', 'something in', 'grit', 'swelling']
eye_syns = ['eye', 'pupil', 'iris']

In [20]:
def tokenize(text):
    tokens = word_tokenize(text)
    tokens = [tok for tok in tokens if tok!='-']
    return tokens

In [21]:
rul_syns = tokenize(re.sub(r", ", " ", rul_csv.syns[0]))
rul_syns

['worker_syns', 'pain_syns', 'eye_syns']

In [22]:
syn_dict.get(rul_syns[0])

['employee',
 ' ip',
 ' worker',
 ' man',
 ' woman',
 ' he',
 ' she',
 ' his',
 ' her',
 ' left',
 ' right',
 ' operator',
 ' operative',
 ' technician',
 ' scaffolder',
 ' team member',
 ' person']

In [125]:
import itertools

c = 1

patterns = []
lhs = []
rhs = []
syn_count = len(rul_syns)

for i in range(syn_count):
    locals()[f'l{i+1}'] = syn_dict.get(rul_syns[i])

iter_prod = itertools.product(l1, l2, l3)

for r in iter_prod: 
    if syn_count == 2:
        pt = f'{r[0].strip()}(.*){r[1].strip()}'
    elif syn_count == 3:
        pt = f'{r[0].strip()}(.*){r[1].strip()}(.*){r[2].strip()}'
    else:
        pt = f'{r[0].strip()}(.*){r[1].strip()}(.*){r[2].strip()}(.*){r[3].strip()}'
    patterns.append(pt)
    lhs.append(r[0].strip())
    rhs.append(r[2].strip())
    c += 1
    
test = pd.DataFrame({'pattern':patterns})    
test['lhs'] = lhs
test['rhs'] = rhs

In [126]:
test

,pattern,lhs,rhs
0,employee(.*)pain(.*)eye,employee,eye
1,employee(.*)pain(.*)pupil,employee,pupil
2,employee(.*)pain(.*)iris,employee,iris
3,employee(.*)discomfort(.*)eye,employee,eye
4,employee(.*)discomfort(.*)pupil,employee,pupil
...,...,...,...
1321,person(.*)swelling(.*)pupil,person,pupil
1322,person(.*)swelling(.*)iris,person,iris
1323,person(.*)swollen(.*)eye,person,eye
1324,person(.*)swollen(.*)pupil,person,pupil


In [127]:
chk_sent = 'the person hurt his right eye'
sum(test['lhs'].apply(lambda x: find_index(chk_sent, x, pos='left')))

936

In [129]:
chk_sent = 'the person hurt his right eye'
split_chk_sent = chk_sent.split()
print(split_chk_sent)
test['found'] = test['pattern'].apply(lambda x: check_presence(x, chk_sent))
test['lhs_index'] = test['lhs'].apply(lambda x: find_index(chk_sent, x, pos='left'))
test['rhs_index'] = test['rhs'].apply(lambda x: find_index(chk_sent, x, pos='right'))
test['span'] = test['rhs_index'] - test['lhs_index']
test

['the', 'person', 'hurt', 'his', 'right', 'eye']


,pattern,lhs,rhs,found,lhs_index,rhs_index,span
0,employee(.*)pain(.*)eye,employee,eye,False,0,6,6
1,employee(.*)pain(.*)pupil,employee,pupil,False,0,0,0
2,employee(.*)pain(.*)iris,employee,iris,False,0,0,0
3,employee(.*)discomfort(.*)eye,employee,eye,False,0,6,6
4,employee(.*)discomfort(.*)pupil,employee,pupil,False,0,0,0
...,...,...,...,...,...,...,...
1321,person(.*)swelling(.*)pupil,person,pupil,False,2,0,-2
1322,person(.*)swelling(.*)iris,person,iris,False,2,0,-2
1323,person(.*)swollen(.*)eye,person,eye,False,2,6,4
1324,person(.*)swollen(.*)pupil,person,pupil,False,2,0,-2


In [132]:
test[test['found'] == True]

,pattern,lhs,rhs,found,lhs_index,rhs_index,span
399,he(.*)hurt(.*)eye,he,eye,True,1,6,5
1257,person(.*)hurt(.*)eye,person,eye,True,2,6,4


In [133]:
test[test['lhs_index'] != 0]

,pattern,lhs,rhs,found,lhs_index,rhs_index,span
390,he(.*)pain(.*)eye,he,eye,False,1,6,5
391,he(.*)pain(.*)pupil,he,pupil,False,1,0,-1
392,he(.*)pain(.*)iris,he,iris,False,1,0,-1
393,he(.*)discomfort(.*)eye,he,eye,False,1,6,5
394,he(.*)discomfort(.*)pupil,he,pupil,False,1,0,-1
...,...,...,...,...,...,...,...
1321,person(.*)swelling(.*)pupil,person,pupil,False,2,0,-2
1322,person(.*)swelling(.*)iris,person,iris,False,2,0,-2
1323,person(.*)swollen(.*)eye,person,eye,False,2,6,4
1324,person(.*)swollen(.*)pupil,person,pupil,False,2,0,-2


In [88]:
chk_sent = 'the person hurt his right hurts eye'
split_chk_sent = chk_sent.split()
print(split_chk_sent)

['the', 'person', 'hurt', 'his', 'right', 'hurts', 'eye']


In [89]:
def find_index(chk_string, keyword, pos='left'):
    try:
        split_chk = chk_string.split()
        #find_index = max(loc for loc, val in enumerate(split_chk) if val == keyword) + 1
        find_index = [loc for loc, val in enumerate(split_chk) if bool(re.search(keyword, val))]
        if pos == 'left':
            find_index = min(find_index)+1
        else:
            find_index = max(find_index)+1
    except:
        print
        find_index = 0
    return(find_index)

find_index(chk_sent, 'person', pos='left')

2

In [14]:
keyword = 'leg'
window = 2
pre_context = "(?:[a-zA-Z'-]+[^a-zA-Z'-]+){0,<window>}<keyword>".replace('<keyword>', str(keyword)).replace('<window>', str(window))
post_context = "<keyword>(?:[^a-zA-Z'-]+[a-zA-Z'-]+){0,<window>}".replace('<keyword>', str(keyword)).replace('<window>', str(window))
all_context = "(?:[a-zA-Z'-]+[^a-zA-Z'-]+){0,<window>}<keyword>(?:[^a-zA-Z'-]+[a-zA-Z'-]+){0,<window>}".replace('<keyword>', str(keyword)).replace('<window>', str(window))
pre_context

"(?:[a-zA-Z'-]+[^a-zA-Z'-]+){0,2}leg"

In [15]:
doc = 'My big fat leg is really broke'
re.findall(pre_context, doc)

['big fat leg']

In [16]:
re.findall(post_context, doc)

['leg is really']

In [23]:
"<keyword> how you doing".replace('<keyword>', str(keyword))

'leg how you doing'